In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../input/income-classification/income_evaluation.csv')
df.head()

In [ ]:
df.shape

In [ ]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df.columns = col_names

df.columns

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.describe(include='all')

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Categorical Variables:
categorical_features = [col for col in df.columns
                       if df[col].dtypes =='object']
categorical_features

In [ ]:
df[categorical_features].nunique()

In [ ]:
df[categorical_features].head()

In [ ]:
for col in categorical_features:
    print(df[col].value_counts())

In [ ]:
for col in categorical_features:
    print(df[col].value_counts()/len(df))

In [ ]:
df['income'].value_counts()

In [ ]:
df['income'].value_counts()/len(df)

In [ ]:
# Lets Visualize frequency distribution of income variable:
f,ax = plt.subplots(figsize=(15,5))
df['income'].value_counts().plot.pie(autopct='%1.1f%%',shadow=True)

In [ ]:
sns.countplot(y='income',data=df)

In [ ]:
# Vizualize income Vs sex variable
sns.countplot(x='income',hue='sex',data=df)

In [ ]:
sns.countplot(x='income',hue='race',data=df)

In [ ]:
df['workclass'].nunique()

In [ ]:
df['workclass'].unique()

In [ ]:
df['workclass'].value_counts()

#### There are 1836 values that are encoded with ?  . So  I will replace them with NaN

In [ ]:

df['workclass'].replace(' ?', np.NaN, inplace=True)

In [ ]:
df['workclass'].value_counts()

In [ ]:
df['workclass'].value_counts().plot.bar(color='orange')
plt.xticks(rotation=30)

In [ ]:
# vizualize workclass Vs Income

sns.countplot(x='workclass',hue='income',data=df)
plt.xticks(rotation=90)

In [ ]:
# Visualize workclass with Sex variable
sns.countplot(y='workclass',hue='sex',data=df)

In [ ]:
# Explore Occupation Variable
df['occupation'].nunique()

In [ ]:
df['occupation'].unique()

In [ ]:
df['occupation'].value_counts()

In [ ]:
# Replace ? with NaN
df['occupation'].replace(' ?',np.NaN,inplace=True)

In [ ]:
df['occupation'].value_counts()

In [ ]:
# Visualize Occupation Variable

df['occupation'].value_counts().plot.bar()

In [ ]:
# Explore native_country
df['native_country'].nunique()

In [ ]:
df['native_country'].unique()

In [ ]:
df['native_country'].value_counts()

In [ ]:
# Replace ? 
df['native_country'].replace(' ?',np.NaN,inplace=True)

In [ ]:
plt.figure(figsize=(20,13))
sns.countplot(x='native_country',data=df)
plt.xticks(rotation=90)


### Checking Missing Values in Categorical Features:

In [ ]:
df[categorical_features].isnull().sum().sort_values(ascending=False)

### Cardinity: Number of labels within a categorical variable

In [ ]:
for col in categorical_features:
    print(col,' contains ',len(df[col].unique()),' labels.')

### Numerical Variables

In [ ]:
numerical_features = [col for col in df.columns
                     if df[col].dtypes !='object']
numerical_features

In [ ]:
df[numerical_features].head()

In [ ]:
df[numerical_features].isnull().sum()

In [ ]:
df['age'].nunique()

In [ ]:
df['age'].head()

In [ ]:
sns.distplot(df['age'],bins=10,color='orange')
plt.show()

In [ ]:
sns.boxplot(df['age'])

we can see that there are many outliers in age variable

In [ ]:
# Relation Between age and income variable
sns.boxplot(x='income',y='age',data=df)

#### Interpretation
As expected, younger people make less money as compared to senior people.

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

### Pairplot

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
sns.pairplot(df,hue='sex')
plt.show()

In [ ]:
### Categorical Variables

df[categorical_features].isnull().sum()

In [ ]:
# Filling missing class with mode

df['workclass'].fillna(df['workclass'].mode()[0],inplace=True)
df['occupation'].fillna(df['occupation'].mode()[0],inplace=True)
df['native_country'].fillna(df['native_country'].mode()[0],inplace=True)

In [ ]:
df[categorical_features].isnull().sum()

In [ ]:
X = df.drop('income',axis=1)
y = df.income

In [ ]:
X.shape,y.shape

In [ ]:
# Encoding
import category_encoders as ce
from sklearn.compose import ColumnTransformer
categorical=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 
                                 'race', 'sex', 'native_country']
encoder = ce.OneHotEncoder()
transformer = ColumnTransformer([('one_hot',encoder,categorical)],
                               remainder='passthrough')
transformed_X = transformer.fit_transform(X)
transformed_X


In [ ]:
pd.DataFrame(transformed_X)

In [ ]:
transformed_X.shape

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler_transformed=scaler.fit_transform(transformed_X)
scaler_transformed.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(scaler_transformed,y,test_size=0.3,random_state=0)

In [ ]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train,y_train)

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
### n_estimators=100
rfc_100 = RandomForestClassifier(n_estimators=100,random_state=0)
rfc_100.fit(X_train,y_train)
y_pred_100 = rfc_100.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred_100)

Until now, I have used all the features given in the model. Now, I will select only the important features, build the model using these features and see its effect on accuracy.

First, I will create the Random Forest model as follows:-

In [ ]:
clf = RandomForestClassifier(n_estimators=100,random_state=0)
clf.fit(X_train,y_train)

In [ ]:
clf.get_params()

In [ ]:
clf.feature_importances_

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
cm_matrix = pd.DataFrame(data=cm,columns=['Actual Positive:1',
                                         'Actual Negative:0'],
                        index=['Predict Positive:1',
                              'Predict Negative:0'])
cm_matrix

In [ ]:
sns.heatmap(cm_matrix,annot=True,fmt='d')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))